# Preprocess Large Holdings File

#### Converting the raw 50+ GB sas file with the holdings complezte data into a sparse python matrix which can be loaded into memory and more important which can be handled more efficiently by different alogorithms. 
#### The logic behind this process is as follows:

Loading data and transforming it into csv file to work with

1. 50+ GB holdings.sas7bdat file containing all the holdings data downloaded directly from wrds using ftp client
2. Converted into csv using sas7bdat_to_csv utility (Link)

Two step process to transform file into sparse matrix
Challenge is to convert from row describing one holding to rows describing the holdings of one fund at one point in time. Aslo it is crucial to keep track of which row of the sparse matrix is which fund at wjich date and which colums are which securities.

3. Open file in python 
4. Parse through file to make two lists. One with all fund/date combinations (using the comination as an ID) and one with all securities.
5. Generate sparse matrix with the dimensions "number of fund/date combinations" x "numer of securities"
6. Parse through large csv file again and fill the percentage_tna (percentage of the fund held in that particular security) number into the right spot of the sparse matrix as determined by two maps based on all fund/date combinations and securities
7. Save final sparse matrix and tables containing information about which row is which fund/date and which column is which security.

TODO

Parsing through csv file could be significantly sped up using something like: https://stackoverflow.com/questions/17444679/reading-a-huge-csv-file

## Import statements

In [1]:
import os
import sys

import feather

import numpy as np
import pandas as pd
from scipy import sparse

## Load File

In [2]:
path = '../data/raw/holdings.csv'

## Parse complete file to get all unique fund/date combinations and stocks

In [3]:
%%time
chunksize = 10 ** 7
unit = chunksize / 184_578_843

reader = pd.read_csv(path,
                     usecols = ['crsp_portno','report_dt',
                                'crsp_company_key','security_name','cusip'],
                     dtype = {'crsp_portno': np.int64,
                              'report_dt': np.int64,
                              'crsp_company_key': np.int64,
                              'security_name': str,
                              'cusip': str},
                     low_memory=False,
                     chunksize=chunksize)

dfList_1 = []
dfList_2 = []

for i, chunk in enumerate(reader):
    temp_df_1 = chunk.loc[:,['crsp_portno','report_dt']].drop_duplicates()
    temp_df_2 = chunk.loc[:,['crsp_company_key','security_name','cusip']].drop_duplicates()
    dfList_1.append(temp_df_1)
    dfList_2.append(temp_df_2)

    print("{:6.2f}%".format(((i+1) * unit * 100)))        

  5.42%
 10.84%
 16.25%
 21.67%
 27.09%
 32.51%
 37.92%
 43.34%
 48.76%
 54.18%
 59.60%
 65.01%
 70.43%
 75.85%
 81.27%
 86.68%
 92.10%
 97.52%
102.94%
108.35%
113.77%
CPU times: user 4min 8s, sys: 1min 4s, total: 5min 12s
Wall time: 5min 18s


In [4]:
df_1 = pd.concat(dfList_1,sort=False)
df_2 = pd.concat(dfList_2,sort=False)

df_1 = df_1.drop_duplicates()
df_2 = df_2.drop_duplicates()

In [5]:
# Generate a unique ID from the portno and the date of a fund/date combination
df_1 = df_1.assign(port_id = ((df_1['crsp_portno'] * 1000000 + df_1['report_dt'])))
df_1 = df_1.rename(columns = {'report_dt':'report_dt_int'})

df_1 = df_1.assign(report_dt = pd.to_timedelta(df_1['report_dt_int'], unit='D') + pd.Timestamp('1960-1-1'))

df_1 = df_1.reset_index(drop = True)
df_1 = (df_1
        .assign(row = df_1.index)
        .set_index('port_id'))

In [6]:
df_2 = df_2.reset_index(drop = True)
df_2 = (df_2
        .assign(col = df_2.index)
        .set_index('crsp_company_key'))

In [7]:
df_1.head(1)

,crsp_portno,report_dt_int,report_dt,row
port_id,,,,
1000001015795,1000001,15795,2003-03-31,0


In [8]:
df_2.head(1)

,security_name,cusip,col
crsp_company_key,,,
3000248,ADVANCED NEUROMODULATION SYS INC,00757T10,0


## Parse complete file to generate data for sparse matrix

In [9]:
%%time
chunksize = 10 ** 7
unit = chunksize / 184_578_843

reader = pd.read_csv(path,
                     usecols = ['crsp_portno','report_dt','crsp_company_key','percent_tna'],
                     dtype = {'crsp_portno': np.int64,
                              'report_dt': np.int64,
                              'crsp_company_key': np.int64,
                              'percent_tna':np.float64},
                     low_memory=False,
                     chunksize=chunksize)

CPU times: user 1.35 ms, sys: 9.57 ms, total: 10.9 ms
Wall time: 10.3 ms


In [10]:
# TODO pd.merge seems to be faster in this case than df.join

In [11]:
%%time
dfList = []

df_1_temp = df_1.loc[:,['row']]
df_2_temp = df_2.loc[:,['col']]

for i, chunk in enumerate(reader):
    temp_df = chunk.dropna()
    temp_df = temp_df.assign(port_id = ((temp_df['crsp_portno'] * 1000000 + temp_df['report_dt'])))
    temp_df.set_index('port_id',inplace=True)
    temp_df = temp_df.join(df_1_temp, how='left')
    temp_df.set_index('crsp_company_key',inplace=True)
    temp_df = temp_df.join(df_2_temp, how='left')
    temp_df = temp_df[['percent_tna','row','col']]
    dfList.append(temp_df)

    print("{:6.2f}%".format(((i+1) * unit * 100)))

  5.42%
 10.84%
 16.25%
 21.67%
 27.09%
 32.51%
 37.92%
 43.34%
 48.76%
 54.18%
 59.60%
 65.01%
 70.43%
 75.85%
 81.27%
 86.68%
 92.10%
 97.52%
102.94%
108.35%
113.77%
CPU times: user 4min 5s, sys: 1min 36s, total: 5min 41s
Wall time: 5min 49s


In [12]:
df_sparse = pd.concat(dfList,sort=False)
df_sparse.reset_index(drop=True,inplace=True)
print(df_sparse.shape)
df_sparse.head(3)

(201112919, 3)


,percent_tna,row,col
0,0.03,339,4255
1,0.09,442,4255
2,0.14,443,4255


## Delete duplicates
All other filters will be applied later but this one has to be done before sparse matrix is created

In [13]:
duplicates_mask = df_sparse.duplicated(['col','row'],keep='last')
df_sparse = df_sparse[~duplicates_mask]

## Check if holdings data makes sense 

In [14]:
merged_data = pd.merge(df_sparse,df_1[['report_dt','row']],how='left',on='row')

In [15]:
date = pd.to_datetime('2016-09-30')

sum_col = (merged_data
           .query('report_dt == @date')
           .groupby(by = ['col'])
           .sum()
           .sort_values('percent_tna',ascending = False))

sum_col.join(df_2.set_index('col'),how='left').head(10)

,percent_tna,row,security_name,cusip
col,,,,
9248,5177.818168,1303244334,USD CASH,NaN
24057,4544.518561,802380589,APPLE INC,03783310
9332,3767.598749,778632575,MICROSOFT CORP,59491810
24032,3530.108884,584119340,AMAZON COM INC,02313510
16343,3165.918936,611808737,FACEBOOK INC,30303M10
25489,2792.909047,639788767,ALPHABET INC,02079K30
582792,2744.169098,33034617,PROSHARES REPO & OTHER INVESTMENTS,NaN
25724,2562.389113,562310958,ALPHABET INC,02079K10
8896,2456.149202,615574004,JPMORGAN CHASE & CO,46625H10


In [16]:
# Seems to make sense. Interestingly Alphabet appears twice. TODO check if two share classes

## Change fund info and security info dfs for future use

In [17]:
df_1 = df_1[['crsp_portno','report_dt','row']].assign(port_id = df_1.index)
df_1.set_index('row',inplace=True)
df_1.sample()

,crsp_portno,report_dt,port_id
row,,,
488529,1026723,2012-06-30,1026723019174


In [18]:
df_2 = df_2.assign(crsp_company_key = df_2.index)
df_2.set_index('col',inplace=True)
df_2.sample()

,security_name,cusip,crsp_company_key
col,,,
815795,BAMLL COMMERCIAL MORTGAGE SECURITIES TRU 15-NO...,054966AW,4018701


## Create sparse matrix

In [19]:
sparse_matrix = sparse.csr_matrix((df_sparse['percent_tna'].values, (df_sparse['row'].values, df_sparse['col'].values)))

In [20]:
# Check if all dimensions match

In [21]:
print('Number of fund/date combinations:        {:12,d}'.format(sparse_matrix.shape[0]))
print('Number of unique securities:             {:12,d}'.format(sparse_matrix.shape[1]))
print('Number of non-zero values in sparse matrix:       {:12,d}'.format(sparse_matrix.getnnz()))
print()
print('Number of rows in fund info df:          {:12,d}'.format(df_1.shape[0]))
print('Number of rows in fund info df:          {:12,d}'.format(df_2.shape[0]))
print()
match_test = (sparse_matrix.shape[0] == df_1.shape[0]) & (sparse_matrix.shape[1] == df_2.shape[0])
print('Everything matches:                              {}'.format(match_test))

Number of fund/date combinations:             839,196
Number of unique securities:                1,621,421
Number of values in sparse matrix:        201,105,481

Number of rows in fund info df:               839,196
Number of rows in fund info df:             1,621,421

Everything matches:                              True


## Save data

#### Sparse matrix containing holdings

In [22]:
path = '../data/interim/holdings'
sparse.save_npz(path, sparse_matrix)

#### Fund/date info

In [23]:
path = '../data/interim/row_info.feather'
feather.write_dataframe(df_1,path)

#### Securities info

In [24]:
path = '../data/interim/col_info.feather'
feather.write_dataframe(df_2,path)